# Audiobooks business case

### Extract the data from the csv

In [1]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt('Audiobooks.csv',delimiter=',')

# The inputs are all columns in the csv, except for the first one [:,0] and the last one [:,-1](targets)
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

### Shuffle the dataset

In [2]:
# Since we will be batching, we want the data to be as randomly spread out as possible
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
shuffled_targets_all = targets_all[shuffled_indices]

### Balance the data sheet

In [9]:
num_one_targets = int(np.sum(shuffled_targets_all))
zero_targets_counter = 0
indices_to_remove = []
for i in range(shuffled_targets_all.shape[0]):
    if shuffled_targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
shuffled_unscaled_inputs_equal_priors = np.delete(shuffled_unscaled_inputs_all, indices_to_remove, axis=0)
shuffled_targets_equal_priors = np.delete(shuffled_targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [10]:
scaled_inputs = preprocessing.scale(shuffled_unscaled_inputs_equal_priors)

In [16]:
print(scaled_inputs.shape
     )

(4474, 10)


In [13]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# Use shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = shuffled_targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [14]:
samples_count = shuffled_inputs.shape[0]

# Assuming we want 80-10-10 distribution of training, validation, and test.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

test_samples_count = samples_count - train_samples_count - validation_samples_count

# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1773.0 3579 0.4953897736797988
228.0 447 0.5100671140939598
236.0 448 0.5267857142857143


### Save the three datasets in *.npz

In [15]:
# Save the three datasets in *.npz.
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)